https://adventofcode.com/2023/day/16

In [1]:
with open("data/16.txt") as fh:
    data = fh.read()

In [2]:
testdata = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
"""

In [3]:
def parse_puzzle(puzzle):
    D = {}
    for r, line in enumerate(puzzle.splitlines()):
        for c, char in enumerate(line):
            D[c - r * 1j] = char
    return D

In [4]:
class Beam:
    def __init__(
        self,
        position: complex,
        direction: complex,
        grid: dict[complex],
        energized: set[complex],
        all_beams: list[complex],
        visited: set[complex],
    ) -> None:
        self.position = position
        self.direction = direction
        self.grid = grid
        self.energized = energized
        self.all_beams = all_beams
        self.visited = visited
        self.onthegrid = True
        self.cycled = False

        self.all_beams.append(self)
        self.land()

    def step(self):
        if not self.onthegrid:
            return
        self.position = self.position + self.direction
        self.land()

    def land(self):
        if self.position not in self.grid:
            self.onthegrid = False
            return
        mirror = self.grid[self.position]
        drxn = self.direction
        if mirror == "/":
            self.direction = {1: 1j, 1j: 1, -1: -1j, -1j: -1}[drxn]
        elif mirror == "\\":
            self.direction = {1: -1j, 1j: -1, -1: 1j, -1j: 1}[drxn]
        elif (mirror == "|" and drxn in (1, -1)) or (
            mirror == "-" and drxn in (1j, -1j)
        ):
            _ = Beam(
                position=self.position,
                direction=self.direction * 1j,
                grid=self.grid,
                energized=self.energized,
                all_beams=self.all_beams,
                visited=self.visited,
            )
            self.direction = self.direction * -1j
        if (self.position, self.direction) in self.visited:
            self.cycled = True
            self.onthegrid = False
        self.energized.add(self.position)
        self.visited.add((self.position, self.direction))

    def __repr__(self):
        return f"Beam(position={self.position}, direction={self.direction})"

In [5]:
def energize_tiles(data, max_steps=1000):
    grid = parse_puzzle(data)
    return energize_grid(grid, max_steps=max_steps)


def energize_grid(grid, startpos=0, startdrxn=1, max_steps=1000):
    energized = set()
    all_beams = []
    visited = set()
    _ = Beam(
        position=startpos,
        direction=startdrxn,
        grid=grid,
        energized=energized,
        all_beams=all_beams,
        visited=visited,
    )

    for _ in range(max_steps):
        if not all_beams:
            break
        for b in all_beams[:]:
            b.step()
            if not b.onthegrid:
                all_beams.remove(b)
    else:
        raise ValueError("incomplete")
    return len(energized)

In [6]:
%time energize_tiles(testdata)

CPU times: user 71 µs, sys: 11 µs, total: 82 µs
Wall time: 84.2 µs


46

In [7]:
%time energize_tiles(data)

CPU times: user 8.56 ms, sys: 105 µs, total: 8.67 ms
Wall time: 8.63 ms


7728

### Part 2

In [8]:
def parse_puzzle_2(puzzle):
    D = {}
    for r, line in enumerate(puzzle.splitlines()):
        for c, char in enumerate(line):
            D[c - r * 1j] = char
    return D, r + 1, c + 1

In [9]:
def max_energize_tiles(puzzle):
    grid, rowcount, colcount = parse_puzzle_2(puzzle)
    max_energized = 0

    maxrow = rowcount - 1
    maxcol = colcount - 1

    for r in range(rowcount):
        startpos = 0 - 1j * r
        startdrxn = 1
        max_energized = max(
            max_energized, energize_grid(grid, startpos=startpos, startdrxn=startdrxn)
        )
        startpos = maxcol - 1j * r
        startdrxn = -1
        max_energized = max(
            max_energized, energize_grid(grid, startpos=startpos, startdrxn=startdrxn)
        )

    for c in range(colcount):
        startpos = c
        startdrxn = -1j
        max_energized = max(
            max_energized, energize_grid(grid, startpos=startpos, startdrxn=startdrxn)
        )
        startpos = c - maxrow * 1j
        startdrxn = 1j
        max_energized = max(
            max_energized, energize_grid(grid, startpos=startpos, startdrxn=startdrxn)
        )

    return max_energized

In [10]:
%time max_energize_tiles(testdata)

CPU times: user 921 µs, sys: 0 ns, total: 921 µs
Wall time: 923 µs


51

In [11]:
%time max_energize_tiles(data)

CPU times: user 1.35 s, sys: 1.43 ms, total: 1.36 s
Wall time: 1.35 s


8061